## QSO & QSO candidates catalog in ten DES SN fields

### All the QSO and QSO candidates come from three catalogs:

__1.Million quasar catalog:__

include 577,146 QSO and 1297,111 quasar candidates with 80% posibility to be QSO
see their web [here](https://heasarc.gsfc.nasa.gov/W3Browse/all/milliquas.html)

__2.DiPompeo'quasar Probabilities catalog:__

include 5,537,436 Pqso > 0.2, here we selected 2,797,507 Pqso > 0.8 quasar candidates
see their [paper](http://adsabs.harvard.edu/abs/2017AJ....153..107T)

__3.Tie's ozDES quasar catalog:__

include 1263 spectroscopically confirmed quasars 
see their [paper](http://adsabs.harvard.edu/abs/2017AJ....153..107T)


### Y3A1 SN Field Variability Catalog

The DES-SN field point sources (RA, DEC) are selected from Eric Morganson's Y3A1 SN Field Variability Catalog.
A sample of 885,181 variable point sources are garthered.
See selection details (internal web) [here](https://opensource.ncsa.illinois.edu/confluence/pages/viewpage.action?spaceKey=DESDM&title=Y3A1+SN+Field+Variability+Catalog)


After crossing match with them with 3 arcsec, we get 2174 QSO and candidates in our final catalog. Note that most Xmatch processes are finished in [TOPCAT](http://www.star.bris.ac.uk/~mbt/topcat/). 


In [ ]:
#make a lite version of million quasar catalog since it is too large, we only need RA and DEC 
import numpy as np
data=np.genfromtxt('/Users/legolason/study/otherwork/tony/mill.txt',dtype='|S32')
for i in range(len(data)):
    data[i,2]=str(data[i,2])[0]

ind =np.where((data[:,2] == 'q') | (data[:,2] == 'Q'))

import pandas as pd
#print data[ind,0:2].reshape(-1,2)
df=pd.DataFrame(data[ind,0:2].reshape(-1,2))
df.to_csv('/Users/legolason/study/otherwork/tony/mill_QSO.txt',index=False,sep=' ',float_format='%12.8f')

In [172]:
#combine million quasar catalog and dip's catalog to a big catalog including 2797509 qso and cnadidates except ozDES
import numpy as np
from astropy.io import fits
import glob, os
import pandas as pd

dip=fits.open('/Users/legolason/study/otherwork/tony/dip_QSO_catalog.fits')
#dip.info()
dd=dip[1].data
ind=np.where(dd['pqso'] > 0.8,True,False )
print np.sum(ind)
ra_dip=dd['ra'][ind]
dec_dip=dd['dec'][ind]

mill=np.genfromtxt('/Users/legolason/study/otherwork/tony/mill_QSO.txt')
ra_mill=mill[0]
dec_mill=mill[1]

ra=np.append(ra_mill,ra_dip)
dec=np.append(dec_mill,dec_dip)

c1 = fits.Column(name='RA',array=ra,format='E')
c2 = fits.Column(name='DEC',array=dec,format='E')
h=fits.BinTableHDU.from_columns([c1, c2])
h.writeto('/Users/legolason/study/otherwork/tony/All_QSO&candicate_ra_dec.fits')

2797507


In [ ]:
#produce the SN_Field_variability source ra dec catalog to match with QSO candidates catalog
import numpy as np
from astropy.io import fits
import glob, os
import pandas as pd

path='/Users/legolason/study/otherwork/tony/SN_varibility/'
os.chdir(path)

ra=np.array([])
dec=np.array([])
for file in zip(glob.glob("*.fits"),range(len(glob.glob("*.fits")))):
    data=fits.open(file[0])[1].data
    #print data['RA'][0:10],data['DEC'][0:10]
    #print len(data)
    ra=np.append(ra,data['ra'])
    dec=np.append(dec,data['dec'])
    
#save data to fits
c1 = fits.Column(name='RA',array=ra,format='E')
c2 = fits.Column(name='DEC',array=dec,format='E')
h=fits.BinTableHDU.from_columns([c1, c2])
h.writeto('/Users/legolason/study/otherwork/tony/SN_field_variability_ra_dec.fits')

In [200]:
#combine SN_field_variability&ALL_QSO_candidates  and ozDES 

data1=fits.open('/Users/legolason/study/otherwork/tony/inozdes_not_in_five.fits')
a=data1[1].data
ra_a=a['col3']
dec_a=a['col4']

data2=fits.open('/Users/legolason/study/otherwork/tony/north_five_SN_field.fits')
b=data2[1].data
ra_b=b['ra_1']
dec_b=b['dec_1']


ra_qso=np.append(ra_a,ra_b)
dec_qso=np.append(dec_a,dec_b)

df=pd.DataFrame({'1RA':ra_qso,
                 '2DEC':dec_qso})
df1=df.sort_values(by='1RA')
#print df1.iloc[:,0]
c1 = fits.Column(name='RA',array=df1.iloc[:,0],format='E')
c2 = fits.Column(name='DEC',array=df1.iloc[:,1],format='E')
h=fits.BinTableHDU.from_columns([c1, c2])
h.writeto('/Users/legolason/study/otherwork/tony/final_SN_field_qso&candidate.fits')